In [1]:
# 加载本代码所依赖的通用外部包
import json
from datasets import load_dataset
import torch
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, AutoTokenizer

In [2]:
# 加载数据集
ccpm_train = load_dataset(path="./data_datasets_CCPM/", data_files= "train.jsonl")

Using custom data configuration data_datasets_CCPM-2e3f28e187745035
Found cached dataset json (/home/djh/.cache/huggingface/datasets/json/data_datasets_CCPM-2e3f28e187745035/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
ccpm_valid = load_dataset(path="./data_datasets_CCPM/", data_files="valid.jsonl")
ccpm_test = load_dataset(path="./data_datasets_CCPM/", data_files="test_public.jsonl")

Using custom data configuration data_datasets_CCPM-dac694de9212fde6
Found cached dataset json (/home/djh/.cache/huggingface/datasets/json/data_datasets_CCPM-dac694de9212fde6/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration data_datasets_CCPM-7d6d99939b9cd24a
Found cached dataset json (/home/djh/.cache/huggingface/datasets/json/data_datasets_CCPM-7d6d99939b9cd24a/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
# 数据集分裂处理
ccpm_train_split = []
for i in range(0, len(ccpm_train["train"])):
    for j in range(0, 4):
        temp_json = {"translation": ccpm_train["train"][i]["translation"]}
        temp_json["choices"] = ccpm_train["train"][i]["choices"][j]
        if j == int(ccpm_train["train"][i]["answer"]):
            temp_json["answer"] = 1
        else:
            temp_json["answer"] = 0
        ccpm_train_split.append(temp_json)

ccpm_valid_split = []
for i in range(0, len(ccpm_valid["train"])):
    for j in range(0, 4):
        temp_json = {"translation": ccpm_valid["train"][i]["translation"]}
        temp_json["choices"] = ccpm_valid["train"][i]["choices"][j]
        if j == int(ccpm_valid["train"][i]["answer"]):
            temp_json["answer"] = 1
        else:
            temp_json["answer"] = 0
        ccpm_valid_split.append(temp_json)

ccpm_test_split = []
for i in range(0, len(ccpm_test["train"])):
    for j in range(0, 4):
        temp_json = {"translation": ccpm_test["train"][i]["translation"]}
        temp_json["choices"] = ccpm_test["train"][i]["choices"][j]
        temp_json["answer"] = 0
        ccpm_test_split.append(temp_json)

In [13]:
# 建模思路一：将翻译和候选诗行进行拼接
ccpm_train_concat = []
for i in range(0, len(ccpm_train_split)):
    temp_json = {"text": ccpm_train_split[i]["translation"] + "=" + ccpm_train_split[i]["choices"],
                 "label": ccpm_train_split[i]["answer"]}
    ccpm_train_concat.append(temp_json)

ccpm_valid_concat = []
for i in range(0, len(ccpm_valid_split)):
    temp_json = {"text": ccpm_valid_split[i]["translation"] + "=" + ccpm_valid_split[i]["choices"],
                 "label": ccpm_valid_split[i]["answer"]}
    ccpm_valid_concat.append(temp_json)

ccpm_test_concat = []
for i in range(0, len(ccpm_test_split)):
    temp_json = {"text": ccpm_test_split[i]["translation"] + "=" + ccpm_test_split[i]["choices"],
                 "label": ccpm_test_split[i]["answer"]}
    ccpm_test_concat.append(temp_json)

In [23]:
# 预处理1-3：
# 1.加载句子标记器以处理字段：text
tokenizer = AutoTokenizer.from_pretrained("thu-cbert-character")

In [24]:
# 2.定义预处理函数
# 用于标记和截断序列，使其不超过模型的最大输入长度：text
def preproccess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
# 3.对整个数据集进行批量化预处理
# 使用数据集映射函数将预处理函数应用于整个数据集
ccpm_train_tokenized = list(map(preproccess_function, ccpm_train_concat))
ccpm_valid_tokenized = list(map(preproccess_function, ccpm_valid_concat))

In [26]:
ccpm_train_tokenized

[{'input_ids': [2, 849, 101, 980, 77, 190, 1041, 539, 97, 1878, 648, 94, 128, 141, 676, 324, 1118, 342, 76, 60, 127, 101, 694, 115, 882, 932, 2193, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]},
 {'input_ids': [2, 849, 101, 980, 77, 190, 1041, 539, 97, 1878, 648, 94, 128, 141, 676, 324, 1118, 342, 76, 60, 2652, 101, 1878, 1619, 882, 932, 2193, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]},
 {'input_ids': [2, 849, 101, 980, 77, 190, 1041, 539, 97, 1878, 648, 94, 128, 141, 676, 324, 1118, 342, 76, 60, 100, 101, 752, 115, 882, 932, 2193, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [

In [27]:
ccpm_train_tokenized2 = []
for i in range(0, len(ccpm_train_concat)):
    temp_json = {"text": ccpm_train_concat[i]["text"],
                 "input_ids": ccpm_train_tokenized[i]["input_ids"],
                 "attention_mask":ccpm_train_tokenized[i]["attention_mask"],
                 "label": ccpm_train_concat[i]["label"]}
    ccpm_train_tokenized2.append(temp_json)

ccpm_valid_tokenized2 = []
for i in range(0, len(ccpm_valid_concat)):
    temp_json = {"text": ccpm_valid_concat[i]["text"],
                 "input_ids": ccpm_valid_tokenized[i]["input_ids"],
                 "attention_mask":ccpm_valid_tokenized[i]["attention_mask"],
                 "label": ccpm_valid_concat[i]["label"]}
    ccpm_valid_tokenized2.append(temp_json)

In [28]:
ccpm_train_tokenized2[0]

{'input_ids': [2,
  849,
  101,
  980,
  77,
  190,
  1041,
  539,
  97,
  1878,
  648,
  94,
  128,
  141,
  676,
  324,
  1118,
  342,
  76,
  60,
  127,
  101,
  694,
  115,
  882,
  932,
  2193,
  3],
 'label': 0}

In [29]:
# 4.文本动态填充到统一长度：最长元素的长度
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [31]:
# 训练
# 使用AutoModelForSequenceClassification加载模型以及语气标签的数量
model = AutoModelForSequenceClassification.from_pretrained("thu-cbert-character", num_labels=2)

Some weights of the model checkpoint at thu-cbert-character were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at thu-cbert-character and a

In [ ]:
from datasets import load_metric
metric = load_metric("accuracy")

In [32]:
# 在训练参数中定义训练超参数
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,

    evaluation_strategy="steps",
    eval_steps=50,
    logging_strategy="steps",
    logging_steps=50,
    save_strategy="steps",
    save_steps=1000,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="tensorboard"
)

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    results = metric.compute(predictions=predictions, references=labels)
    return results

In [33]:
# 将训练参数与模型、数据集、分词器和数据整理器一起传递给训练器
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ccpm_train_tokenized2,
    eval_dataset=ccpm_valid_tokenized2,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [34]:
# 微调模型
trainer.train()

/home/djh/anaconda3/envs/CL5/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 97992
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 6125
  Number of trainable parameters = 108031490
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 12.00 MiB (GPU 0; 7.79 GiB total capacity; 2.83 GiB already allocated; 22.81 MiB free; 3.07 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [30]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 10880
  Batch size = 16


{'eval_loss': 0.5118282437324524,
 'eval_runtime': 4.7525,
 'eval_samples_per_second': 2289.301,
 'eval_steps_per_second': 143.081,
 'epoch': 1.0}